## DriverNet: autonomous car

### Import all the libraries we need

In [1]:
import csv
import sklearn
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Conv2D

/home/agonzale34/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array
Using TensorFlow backend.


### Loading the dataset

In [9]:
angle_correction = 0.2
data_folder = './training-data/'

car_images = []
car_angles = []

# read all the lines in the csv file
with open(data_folder + 'driving_main_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        angle_center = float(line[3])
        angle_left = angle_center + angle_correction
        angle_right = angle_center - angle_correction
        car_images = car_images + [line[0], line[1], line[2]]
        car_angles = car_angles + [angle_center, angle_left, angle_right]
        
print('Dataset CSV loaded successfully length:', len(car_images))

Dataset CSV loaded successfully length: 7362


### For using fit generator

In [17]:
# creating the train a validation data
train_images, valid_images = train_test_split(car_images, test_size=0.2)
train_angles, valid_angles = train_test_split(car_angles, test_size=0.2)


# def add_image_list(img_list, in_img_path):
#     img_name_path = data_folder + 'IMG/' + in_img_path.split('/')[-1]
#     img_list.append(plt.imread(img_name_path))


# define the generator
def generator(in_images, in_angles, batch_size=32):
    n_lines = len(in_images)
    # Loop forever so the generator never terminates
    while 1:
        for offset in range(0, n_lines, batch_size):
            batch_images = in_images[offset:offset + batch_size]
            batch_angles = in_angles[offset:offset + batch_size]

            images = []
            for batch_image in batch_images:    
                img_name_path = data_folder + 'IMG/' + batch_image.split('/')[-1]
                images.append(plt.imread(img_name_path))
                
            X_train = np.array(images)
            y_train = np.array(batch_angles)
            yield sklearn.utils.shuffle(X_train, y_train)


print('Dataset loaded successfully')
print('Number of features:', len(car_images))

Dataset loaded successfully
Number of features: 7362


### DriverNet model

In [18]:
RELU = 'relu'

model = Sequential()
# Normalize the images
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Conv2D(24, kernel_size=(5, 5), strides=(2, 2), activation=RELU))
model.add(Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation=RELU))
model.add(Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation=RELU))
model.add(Conv2D(64, kernel_size=(3, 3), activation=RELU))
model.add(Conv2D(64, kernel_size=(3, 3), activation=RELU))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

print('DriverNet created successfully')  

DriverNet created successfully


### Training DriverNet

In [19]:
model.compile(loss='mse', optimizer='adam')

# create the generators
train_generator = generator(train_images, train_angles, batch_size=5)
valid_generator = generator(valid_images, valid_angles, batch_size=5)

model.fit_generator(
    train_generator, steps_per_epoch=len(train_angles)/5, epochs=5, 
    validation_data=valid_generator, validation_steps=len(valid_angles)/5,
    max_queue_size=200, workers=4, use_multiprocessing=True
)

#model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

# Save the model
model.save('model.h5')
print('DriverNet trained and saved successfully')  

Epoch 1/5
 361/1177 [========>.....................] - ETA: 29s - loss: 0.0372

KeyboardInterrupt: 